In [2]:
#필요한 라이브러리 임포트
 
import os
import time
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split



In [ ]:
# 랜덤 시드 고정: 결과 재현성을 위해 시드를 고정

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED) # 해시 시드 고정
random.seed(SEED) # 파이썬 랜덤 시드 고정
np.random.seed(SEED) # NumPy 랜덤 시드 고정
torch.manual_seed(SEED) # PyTorch CPU 시드 고정
torch.cuda.manual_seed(SEED) # PyTorch CUDA 시드 고정
torch.cuda.manual_seed_all(SEED) # 모든 GPU에 대한 PyTorch CUDA 시드 고정
torch.backends.cudnn.benchmark = True # CUDA의 성능 최적화

In [3]:
# 데이터셋 클래스를 정의

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values # CSV 파일에서 데이터 로드
        self.path = path # 이미지 파일 경로
        self.transform = transform # 데이터 변환 함수

    def __len__(self):
        return len(self.df) # 데이터셋의 길이 반환

    def __getitem__(self, idx):
        name, target = self.df[idx] # 이미지 이름과 타겟 레이블 추출
        img = np.array(Image.open(os.path.join(self.path, name))) # 이미지 로드
        if self.transform:
            img = self.transform(image=img)['image'] # 변환 적용
        return img, target # 이미지와 레이블 반환



In [4]:
# 이미지 크기 및 변환 설정
img_size = 260

# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])


In [5]:
# 데이터셋 정의
full_dataset = ImageDataset("data2/train2.csv", "data2/train", transform=trn_transform)

# 데이터셋 크기 및 분할
dataset_size = len(full_dataset)
train_ratio = 0.8
train_size = int(train_ratio * dataset_size)
val_size = dataset_size - train_size

# 전체 인덱스를 생성하고 섞기
indices = list(range(dataset_size))
random.seed(42)  # 랜덤 시드 고정
random.shuffle(indices)

# 섞인 인덱스를 사용하여 데이터셋 분할
train_indices = indices[:train_size]
val_indices = indices[train_size:]

trn_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_dataset = torch.utils.data.Subset(full_dataset, val_indices)

# DataLoader 정의
BATCH_SIZE = 32
num_workers = 0

trn_loader = DataLoader(trn_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# 데이터셋 크기 출력
print("Full dataset size:", dataset_size)
print("Training dataset size:", len(trn_dataset))
print("Validation dataset size:", len(val_dataset))



Full dataset size: 147616
Training dataset size: 118092
Validation dataset size: 29524


In [6]:
# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 설정
model_name = 'resnet50'
model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)

# 손실 함수 및 옵티마이저 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)


In [7]:
# 에폭당 학습을 위한 함수

def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train() # 모델을 학습 모드로 설정
    train_loss = 0
    preds_list = [] # 예측 결과 리스트 초기화
    targets_list = [] # 타겟 리스트 초기화

    pbar = tqdm(loader) # 진행 상황을 표시하기 위한 tqdm
    for image, targets in pbar:
        image = image.to(device) # 이미지 텐서를 지정한 장치로 이동
        targets = targets.to(device) # 타겟 텐서를 지정한 장치로 이동

        model.zero_grad(set_to_none=True) # 그래디언트 초기화

        preds = model(image) # 모델 예측
        loss = loss_fn(preds, targets) # 손실 계산
        loss.backward() # 역전파
        optimizer.step() # 최적화 단계

        train_loss += loss.item() # 손실 누적
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy()) # 예측 결과 추가
        targets_list.extend(targets.detach().cpu().numpy()) # 타겟 추가

        pbar.set_description(f"Loss: {loss.item():.4f}") # 진행 바에 손실 출력

    train_loss /= len(loader) # 평균 손실 계산
    train_acc = accuracy_score(targets_list, preds_list) # 정확도 계산
    train_f1 = f1_score(targets_list, preds_list, average='macro') # F1 점수 계산

    ret = {
        "train_loss": train_loss, # 평균 손실
        "train_acc": train_acc, # 정확도
        "train_f1": train_f1, # F1 점수
    }

    return ret # 결과 반환

def validation(model, val_loader, loss_fn):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []
    
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            loss = loss_fn(val_outputs, val_labels)
            val_loss += loss.item()
            preds_list.extend(val_outputs.argmax(dim=1).cpu().numpy())
            targets_list.extend(val_labels.cpu().numpy())
    
    val_loss /= len(val_loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')
    
    return val_loss, val_acc, val_f1



In [8]:
EPOCHS = 100
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(EPOCHS):
    train_results = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)
    val_loss, val_accuracy, val_f1 = validation(model, val_loader, loss_fn)

    print(f"Epoch [{epoch + 1}/{EPOCHS}], "
          f"Train Loss: {train_results['train_loss']:.4f}, "
          f"Train Acc: {train_results['train_acc']:.4f}, "
          f"Train F1: {train_results['train_f1']:.4f}, "
          f"Val Loss: {val_loss:.4f}, "
          f"Val Acc: {val_accuracy:.4f}, "
          f"Val F1: {val_f1:.4f}")
    
    # 조기 종료 조건
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"./model_baseline_best.pt")
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print("Early stopping")
        break


    
# 마지막 모델 저장
torch.save(model.state_dict(), f"./model_baseline_last.pt")

Loss: 0.0140: 100%|██████████| 3691/3691 [15:20<00:00,  4.01it/s]


Epoch [1/100], Train Loss: 0.1745, Train Acc: 0.9379, Train F1: 0.9377, Val Loss: 0.0532, Val Acc: 0.9810, Val F1: 0.9812


Loss: 0.0717: 100%|██████████| 3691/3691 [13:59<00:00,  4.40it/s]


Epoch [2/100], Train Loss: 0.0460, Train Acc: 0.9839, Train F1: 0.9839, Val Loss: 0.0502, Val Acc: 0.9835, Val F1: 0.9837


Loss: 0.0172: 100%|██████████| 3691/3691 [13:53<00:00,  4.43it/s]


...

Epoch [11/100], Train Loss: 0.0082, Train Acc: 0.9972, Train F1: 0.9972, Val Loss: 0.0188, Val Acc: 0.9938, Val F1: 0.9939


Loss: 0.0001: 100%|██████████| 3691/3691 [13:47<00:00,  4.46it/s]


Epoch [12/100], Train Loss: 0.0074, Train Acc: 0.9976, Train F1: 0.9976, Val Loss: 0.0248, Val Acc: 0.9923, Val F1: 0.9924


Loss: 0.0012: 100%|██████████| 3691/3691 [10:40<00:00,  5.77it/s]


Early stopping


In [9]:
# 테스트 데이터셋 로드
tst_dataset = ImageDataset("data2/sample_submission.csv", "data2/test", transform=tst_transform)
tst_loader = DataLoader(tst_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# 베스트 모델 로드
model.load_state_dict(torch.load("./model_resnet50_best.pt"))  # 저장한 모델 로드
model.to(device)  # 모델을 장치로 이동

# 예측 리스트 초기화
preds_list = []

# 모델을 평가 모드로 설정
model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 결과 DataFrame 생성
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

# 샘플 제출 파일 로드 및 확인
sample_submission_df = pd.read_csv("data2/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

# 예측 결과를 CSV 파일로 저장
pred_df.to_csv("pred_r50_best.csv", index=False)

100%|██████████| 99/99 [00:10<00:00,  9.41it/s]


In [10]:
# 테스트 데이터셋 로드
tst_dataset = ImageDataset("data2/sample_submission.csv", "data2/test", transform=tst_transform)
tst_loader = DataLoader(tst_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# 베스트 모델 로드
model.load_state_dict(torch.load("./model_resnet50_last.pt"))  # 저장한 모델 로드
model.to(device)  # 모델을 장치로 이동

# 예측 리스트 초기화
preds_list = []

# 모델을 평가 모드로 설정
model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)

    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 결과 DataFrame 생성
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

# 샘플 제출 파일 로드 및 확인
sample_submission_df = pd.read_csv("data2/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

# 예측 결과를 CSV 파일로 저장
pred_df.to_csv("pred_r50_last.csv", index=False)

100%|██████████| 99/99 [00:12<00:00,  7.69it/s]
